In [1]:
!pip install -q -U langchain langchain_huggingface transformers bitsandbytes accelerate langchain_community gradio pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 

In [2]:
import torch
from transformers import BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import gradio as gr
import pandas as pd
import json

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# model_llama = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", device_map="auto",quantization_config=quantization_config, )
# tokenizer_llama = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

model_mistral = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", device_map="auto",quantization_config=quantization_config, )
tokenizer_mistral = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/138k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# terminators_llama = [
#     tokenizer_llama.eos_token_id,
#     tokenizer_llama.convert_tokens_to_ids("<|eot_id|>")
# ]

# pipeline_llama = pipeline(
#         "text-generation",
#         model=model_llama,
#         tokenizer=tokenizer_llama,
#         use_cache=True,
#         device_map="auto",
#         max_new_tokens=256,
#         do_sample=True,
#         top_k=5,
#         num_return_sequences=1,
#         eos_token_id=terminators_llama,
# )

In [44]:
pipeline_mistral = pipeline(
        "text-generation",
        model=model_mistral,
        tokenizer=tokenizer_mistral,
        use_cache=True,
        device_map="auto",
        max_length=2500,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer_mistral.eos_token_id,
        pad_token_id=tokenizer_mistral.eos_token_id,
)

In [45]:
llm = HuggingFacePipeline(pipeline=pipeline_mistral) # change pipeline according to LLM

In [46]:
template_mistral = """<s>[INST] Extract email signature information and structure it into a JSON format.
You have to give valid JSON with extracted NERs without any additional explanation.
Possible keys for extracted fields: first_name, last_name, company, position, phone, email, website, linkedin, twitter etc.
Structure the JSON output so that the extracted fields are all contained within a single object __sig__ .
Do not include not found values in JSON. Do not include academic degrees.
If there is several phone numbers, seperate them by comma and space under key `phone`.

Always start JSON with ```json and end with ```

Email signature is given below :
{signature} [/INST] </s>
"""

# template_llama = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

# Extract email signature information and structure it into a JSON format.
# You have to give valid JSON with extracted NERs without any additional explanation.
# Structure the JSON output so that the extracted fields (like first name, last name, company, position, etc.) are all contained within a single object __sig__ .
# Always start JSON with ```json and end with ```<|eot_id|><|start_header_id|>user<|end_header_id|>

# Email signature is given below :
# {signature}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

In [8]:
def generate_response(template, signature):
    """
    Asks LLM for response using a given template and email signature.

    Args:
        template (str): The template to be used for generating the response from a specific LLM.
        signature (dict): email containing signature information.

    Returns:
        str: Plain response from LLM.
    """
    prompt = PromptTemplate(template=template, input_variables=["signature"])
    llm_chain = prompt | llm
    response = llm_chain.invoke({"signature":signature})
    return response

def grade_signatures(llm_response, ideal):
    """
    Grades the LLM response signature JSON against the ideal response JSON.

    Args:
        llm_response (dict): The JSON response from the language model.
        ideal (dict): The ideal JSON structure to compare against.

    Returns:
        tuple: A tuple containing:
            - score (int): The total score based on key presence and value matches.
            - total_possible_points (int): The total possible points for correct keys and values.
            - details (dict): A dictionary detailing the discrepancies:
                - "missing_keys" (list): Keys that are in the ideal JSON but missing in the response.
                - "extra_keys" (list): Keys that are in the response but not in the ideal JSON.
                - "value_mismatches" (dict): Keys that have mismatched values, with expected and found values.
    """
    total_possible_points = 0
    score = 0
    details = {
        "missing_keys": [],
        "extra_keys": [],
        "value_mismatches": {}
    }

    # Ensure both are dictionaries
    if not isinstance(llm_response, dict) or not isinstance(ideal, dict):
        return 0, "One of the inputs is not a dictionary"

    # Compare keys
    llm_response_keys = set(llm_response.keys())
    ideal_keys = set(ideal.keys())

    total_possible_points += len(ideal_keys)

    if llm_response_keys != ideal_keys:
        details["missing_keys"] = list(ideal_keys - llm_response_keys)
        details["extra_keys"] = list(llm_response_keys - ideal_keys)

    # Add points for correct keys
    score += len(ideal_keys - set(details["missing_keys"]))

    # Compare values
    for key in ideal_keys:
        if key in llm_response:
            total_possible_points += 1
            if llm_response[key] == ideal[key]:
                score += 1
            else:
                details["value_mismatches"][key] = {
                    "expected": ideal[key],
                    "found": llm_response[key]
                }

    return score, total_possible_points, details

def extract_json(template, email):
    """
    Extracts JSON content from an LLM response.

    Args:
        template (str): The template used to generate the response from a particular LLM.
        email (str): The email which contains signature infromation.

    Returns:
        dict: The extracted JSON content.
    """
    response = generate_response(template, email)

    part_with_json = response.replace('\n', '').split("```json")[-1] # split by the place where json begins with ```json

    json_str = part_with_json.split("```")[0] # split by ``` to get only json with brackets

    return json.loads(json_str)

In [47]:
def gradio_interface(email):
    response = extract_json(template_mistral, email)
    return json.dumps(response, indent=4, ensure_ascii=False)

iface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(lines=10, placeholder="Enter your email here..."),
    outputs="text",
    title="Email Signature Generator"
)

iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://825effc92db627968a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [16]:
email_signatures = ["""Hi Mike,
I just wanted to follow up on the project we discussed during our meeting last week and see if you had any updates or needed any assistance from me.
Jane Doe
Sales & Marketing Director
ABC Company 3373 Gregory Lane, 40601 Frankfurt
t: (800) 555-1133
e: jane.doe@example.com
www.example.com""",

"""Hello Maria,
Can we schedule a meeting to discuss the marketing strategy?

Best,
Tom
tom@example.com""",

"""Hi,
Please review the attached document for our next meeting.

Best,
Богдана Нікітченко
NLP Engineer
XYZ Ltd.
Phone: (555) 123-4567
jane.doe@xyz.com
www.xyz.com""",

"""Hi Ostap,
There is an update on project timelines. We have to schedule a meeting to discuss it. What is the most suitable time for you?

Best,
Sarah Lee
Product Manager
Beta Inc.
Direct: (111) 222-3333
Mobile: (444) 555-6666
Office: (777) 888-9999
sarah.lee@beta.com""",

"""Hi Everyone,
Don't forget to check our social media for updates.

Thanks,
David Green
HR Manager
Gamma Ltd.
david.green@gammaltd.com
LinkedIn: linkedin.com/in/davidgreen
Twitter: @davidgreen""",

"""Hi Team,
I hope all is well. I wanted to follow up on the project we discussed last week.
Can you confirm if the proposed deadlines are still feasible for you? Please let me know if there are any adjustments needed or if you require any support from my side.

Best wishes,
Rebecca Adams, Ph.D.
Director of Research | Epsilon Technologies
rebecca.adams[at]epsilon.com | (333) 777-8888""",

"""Hallo Team,
Die Besprechungsnotizen sind im Anhang.

Mit freundlichen Grüßen,
Anna Müller
Projektleiterin
Theta GmbH
Tel: +49 89 1234567
anna.mueller@thetagmbh.de""",

"""Hi there,
Please review the document and provide feedback.

Best,
Dr. Robert Taylor, MD
CEO, Omega Medical
Office: (555) 123-4567
rtaylor@omegamed.com
www.omegamed.com""",

"""Hi All,
Here are my contact details for the upcoming project.

Warm regards,
Michael Brown
VP of Engineering
Acme Corp.
michael.brown@acmecorp.com
mobile: (987) 654-3210
direct: (123) 456-7890
skype: michael.brown.acme"""
                    ]

In [17]:
expected_jsons = [{
    "__sig__": {
        "first_name": "Jane",
        "last_name": "Doe",
        "company": "ABC Company",
        "position": "Sales & Marketing Director",
        "phone": "(800) 555-1133",
        "email": "jane.doe@example.com",
        "website": "www.example.com"
    }
},
{
    "__sig__": {
        "first_name": "Tom",
        "email": "tom@example.com"
    }
},
{
    "__sig__": {
        "first_name": "Богдана",
        "last_name": "Нікітченко",
        "position": "NLP Engineer",
        "company": "XYZ Ltd.",
        "phone": "(555) 123-4567",
        "email": "jane.doe@xyz.com",
        "website": "www.xyz.com"
    }
},
{
    "__sig__": {
        "first_name": "Sarah",
        "last_name": "Lee",
        "position": "Product Manager",
        "company": "Beta Inc.",
        "phone": "(111) 222-3333, (444) 555-6666, (777) 888-9999",
        "email": "sarah.lee@beta.com"
    }
},

{
    "__sig__": {
        "first_name": "David",
        "last_name": "Green",
        "position": "HR Manager",
        "company": "Gamma Ltd.",
        "email": "david.green@gammaltd.com",
        "linkedin": "linkedin.com/in/davidgreen",
        "twitter": "@davidgreen"
    }
},
{
    "__sig__": {
        "first_name": "Rebecca",
        "last_name": "Adams",
        "position": "Director of Research",
        "company": "Epsilon Technologies",
        "phone": "(333) 777-8888",
        "email": "rebecca.adams@epsilon.com"
    }
},
{
    "__sig__": {
        "first_name": "Anna",
        "last_name": "Müller",
        "position": "Projektleiterin",
        "company": "Theta GmbH",
        "phone": "+49 89 1234567",
        "email": "anna.mueller@thetagmbh.de"
    }
},
{
    "__sig__": {
        "first_name": "Robert",
        "last_name": "Taylor",
        "position": "CEO",
        "company": "Omega Medical",
        "phone": "(555) 123-4567",
        "email": "rtaylor@omegamed.com",
        "website": "www.omegamed.com"
    }
},
{
    "__sig__": {
        "first_name": "Michael",
        "last_name": "Brown",
        "position": "VP of Engineering",
        "company": "Acme Corp.",
        "email": "michael.brown@acmecorp.com",
        "phone": "(987) 654-3210, (123) 456-7890",
        "skype": "michael.brown.acme"
    }
}
                  ]

In [51]:
# initialize DataFrame to store results

data = {
    "score": [],
    "details_of_mismatches": [],
    "template": [],
    "ideal_output": [],
    "llm_output": []
}

df = pd.DataFrame(data)

In [48]:
for email_signature, ideal_response in zip(email_signatures, expected_jsons):
  llm_response = extract_json(template_mistral, email_signature)
  llm_respone_sig = llm_response.get("__sig__", {})
  ideal_sig = ideal_response.get("__sig__", {})

  score, total_possible_points, comparison_details = grade_signatures(llm_respone_sig, ideal_sig)
  percentage = (score / total_possible_points) * 100

  new_data = {
      "score": [percentage],
      "details_of_mismatches": [comparison_details],
      "template": [template_mistral],
      "ideal_output": [ideal_response],
      "llm_output": [llm_response]
  }

  new_df = pd.DataFrame(new_data)

  df = pd.concat([df, new_df], ignore_index=True)

In [ ]:
df.to_csv("mistral.csv")